In [ ]:
!apt-get install openjdk-8-jdk-headless

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u312-b07-0ubuntu1~18.04 [28.2 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jdk-headless amd64 8u312-b07-0ubuntu1~18.

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

--2022-05-19 22:57:32--  https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M  24.4MB/s    in 10s     

2022-05-19 22:57:43 (25.0 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [ ]:
!tar xf /content/spark-3.2.1-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.2.1-bin-hadoop2.7'

In [ ]:
import pyspark
import numpy as np
import pandas as pd

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('data').getOrCreate()
data = spark.read.csv('/content/combined-data.csv', sep=',', header=True, inferSchema=True, nullValue='NA')
data.show(5)

+------+-------------+---------+------------+----------------+----------+-----------+---------+
|userId|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
|   937|         5652|        1|     android|              39|         0|          1|      1.0|
|  1623|         5659|        1|      iphone|             129|         9|          1|     10.0|
|    83|         5661|        1|     android|             102|        14|          1|      5.0|
|   121|         5665|        1|     android|              39|         4|          1|      3.0|
|   462|         5666|        1|     android|              90|        10|          1|      3.0|
+------+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



In [ ]:
data = data.drop('userId')
data.show(5)

+-------------+---------+------------+----------------+----------+-----------+---------+
|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|
+-------------+---------+------------+----------------+----------+-----------+---------+
|         5652|        1|     android|              39|         0|          1|      1.0|
|         5659|        1|      iphone|             129|         9|          1|     10.0|
|         5661|        1|     android|             102|        14|          1|      5.0|
|         5665|        1|     android|              39|         4|          1|      3.0|
|         5666|        1|     android|              90|        10|          1|      3.0|
+-------------+---------+------------+----------------+----------+-----------+---------+
only showing top 5 rows



In [ ]:
# Create 'label' column indicating whether average price (1) or not(0)
data = data.withColumn('label', (data.avg_price >= 5).cast('integer'))
data.show(5)

+-------------+---------+------------+----------------+----------+-----------+---------+-----+
|userSessionId|teamLevel|platformType|count_gameclicks|count_hits|count_buyId|avg_price|label|
+-------------+---------+------------+----------------+----------+-----------+---------+-----+
|         5652|        1|     android|              39|         0|          1|      1.0|    0|
|         5659|        1|      iphone|             129|         9|          1|     10.0|    1|
|         5661|        1|     android|             102|        14|          1|      5.0|    1|
|         5665|        1|     android|              39|         4|          1|      3.0|    0|
|         5666|        1|     android|              90|        10|          1|      3.0|    0|
+-------------+---------+------------+----------------+----------+-----------+---------+-----+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StringIndexer


In [ ]:
indexer = StringIndexer(inputCol='platformType', outputCol='platformType_idx')


In [ ]:
indexer_model = indexer.fit(data)
data_indexed = indexer_model.transform(data)

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    'platformType_idx', 'teamLevel','count_gameclicks','count_buyId',
], outputCol='features')

In [ ]:
data_assembled = assembler.transform(data_indexed)

In [ ]:
data_assembled.select('features', 'avg_price').show(5, truncate=False)

+-------------------+---------+
|features           |avg_price|
+-------------------+---------+
|[1.0,1.0,39.0,1.0] |1.0      |
|[0.0,1.0,129.0,1.0]|10.0     |
|[1.0,1.0,102.0,1.0]|5.0      |
|[1.0,1.0,39.0,1.0] |3.0      |
|[1.0,1.0,90.0,1.0] |3.0      |
+-------------------+---------+
only showing top 5 rows



In [ ]:
# Split into training and test sets in a 80:20 ratio
data_train, data_test = data_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = data_train.count() / data_assembled.count()
print(training_ratio)

0.8015591778880227


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier



In [ ]:
data_train.head()

Row(userSessionId=5659, teamLevel=1, platformType='iphone', count_gameclicks=129, count_hits=9, count_buyId=1, avg_price=10.0, label=1, platformType_idx=0.0, features=DenseVector([0.0, 1.0, 129.0, 1.0]))

In [ ]:
# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(data_train)


In [ ]:
# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(data_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|0    |0.0       |[0.7843137254901961,0.21568627450980393]|
|0    |0.0       |[0.7843137254901961,0.21568627450980393]|
|1    |1.0       |[0.0784313725490196,0.9215686274509803] |
|1    |1.0       |[0.0784313725490196,0.9215686274509803] |
|1    |0.0       |[0.75,0.25]                             |
+-----+----------+----------------------------------------+
only showing top 5 rows



In [ ]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   32|
|    0|       0.0|  129|
|    1|       1.0|  112|
|    0|       1.0|    7|
+-----+----------+-----+

0.8607142857142858
